Since our data is time series, we must:
    1. Perform 31-day sliding window min-max scaling
    2. Create a 0-1 binary predictor variables on whether the coin was pumped
For __OHLCV__ Data

In [1]:
import os, sys
import pandas as pd 
import numpy as np
import requests 
from datetime import datetime, date, time, timedelta
import pytz
import matplotlib.pyplot as plt
import matplotlib.dates as mdate
from mpl_finance import candlestick_ohlc
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.ticker as mticker
import pickle
pd.set_option('display.max_columns', 999)
%load_ext autoreload
%matplotlib inline

In [2]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [3]:
print(os.getcwd())

/Users/hn/Documents/USC Others/ISI Research/PumpCoin/CrytoCompare/All_Pump_Analysis/Before_Announcement_Analysis


# PART 1: SLIDING WINDOW NORMALIZATION

### GENERAL PROCEDURE FOR WINDOW NORMALIZATION
1. Iterate through row
2. For each of the group of variables (x_d0 to x_d31), apply the function
3. This returns an aray, keep appending them until we have the entire array
4. Append the un-scaled features, and now have a complete row
5. Keep stackign them to a blank array

# DAILY 

In [61]:
d = pd.read_pickle('ohlcv_data_training_d.pkl').reset_index(drop=True)

In [62]:
print("Original OHLCV Data has shape: ", d.shape)

Original OHLCV Data has shape:  (43506, 200)


In [63]:
cd = pd.read_pickle("ohlcv_dict_d.pkl")

In [64]:
def minmax(x, varlist):
    x_t = x.T
    scaler = MinMaxScaler()
    scaler.fit(x_t)
    r = scaler.transform(x_t)
    return pd.DataFrame(r.T, columns=varlist)

In [65]:
#create a list of features
ohlcv_feat = pd.read_pickle("ohlcv_features_d.pkl")
len(ohlcv_feat)

160

In [66]:
orig_cols = list(d.columns)
d_scaled = d[cd['core']]
d_temp = np.zeros(shape=(1,d.shape[1]))
print("d_temp", d_temp.shape)

for k,v in cd.items():
    if k != 'core':
        var = cd[k] 
        d_temp = minmax(d[var], var)
        print(d_temp.shape, type(d_temp))
        d_scaled = pd.concat((d_scaled,d_temp),axis=1)

d_temp (1, 200)
(43506, 32) <class 'pandas.core.frame.DataFrame'>
(43506, 32) <class 'pandas.core.frame.DataFrame'>
(43506, 32) <class 'pandas.core.frame.DataFrame'>
(43506, 32) <class 'pandas.core.frame.DataFrame'>
(43506, 32) <class 'pandas.core.frame.DataFrame'>
(43506, 32) <class 'pandas.core.frame.DataFrame'>


In [22]:
#Verify that the scaled dataset has the same shape as the original
d_scaled.shape

(43506, 200)

In [23]:
d_scaled['pumped_yn'] = 0
d_scaled.loc[d_scaled.currency == d_scaled.coin_name, 'pumped_yn'] = 1

###  PLOT THE VARIABLE GROUPS TO MAKE SURE THE SCALED ARE THE SAME
Check that the plots are essential the same, just with range normalized between and 0 and 1

In [57]:
#Select a random show
cnt = 0
#Select a random show
r = d_scaled[(d_scaled.pumped_yn == 0) & (d_scaled.currency=="PAI") & ()].sample(1)
s = d.loc[r.index]
s2 = r
fig = plt.figure(figsize=(10,70))
print(r.currency)

for k,v in cd.items():
    if k != 'core':
        var = cd[k]
        ax1 = fig.add_subplot(33,2,cnt*2+1)
        ax1 = s.iloc[0][var].plot()
        ax1.set_xticklabels([])
        ax1.set_xlabel(k)
        ax2 =fig.add_subplot(33,2,cnt*2+2)
        ax2 = s2.iloc[0][var].plot()
        ax2.set_xticklabels([])
        ax2.set_xlabel(k)
        cnt+=1

ValueError: operands could not be broadcast together with shapes (43506,) (0,) 

In [12]:
d_scaled.head()

,channelLink,channelTitle,currency,exchange,timestamp,signalTime,pumptime_edited,coin_name,open_d0,open_d1,open_d2,open_d3,open_d4,open_d5,open_d6,open_d7,open_d8,open_d9,open_d10,open_d11,open_d12,open_d13,open_d14,open_d15,open_d16,open_d17,open_d18,open_d19,open_d20,open_d21,open_d22,open_d23,open_d24,open_d25,open_d26,open_d27,open_d28,open_d29,open_d30,open_d31,high_d0,high_d1,high_d2,high_d3,high_d4,high_d5,high_d6,high_d7,high_d8,high_d9,high_d10,high_d11,high_d12,high_d13,high_d14,high_d15,high_d16,high_d17,high_d18,high_d19,high_d20,high_d21,high_d22,high_d23,high_d24,high_d25,high_d26,high_d27,high_d28,high_d29,high_d30,high_d31,low_d0,low_d1,low_d2,low_d3,low_d4,low_d5,low_d6,low_d7,low_d8,low_d9,low_d10,low_d11,low_d12,low_d13,low_d14,low_d15,low_d16,low_d17,low_d18,low_d19,low_d20,low_d21,low_d22,low_d23,low_d24,low_d25,low_d26,low_d27,low_d28,low_d29,low_d30,low_d31,close_d0,close_d1,close_d2,close_d3,close_d4,close_d5,close_d6,close_d7,close_d8,close_d9,close_d10,close_d11,close_d12,close_d13,close_d14,close_d15,close_d16,close_d17,close_d18,close_d19,close_d20,close_d21,close_d22,close_d23,close_d24,close_d25,close_d26,close_d27,close_d28,close_d29,close_d30,close_d31,volumefrom_d0,volumefrom_d1,volumefrom_d2,volumefrom_d3,volumefrom_d4,volumefrom_d5,volumefrom_d6,volumefrom_d7,volumefrom_d8,volumefrom_d9,volumefrom_d10,volumefrom_d11,volumefrom_d12,volumefrom_d13,volumefrom_d14,volumefrom_d15,volumefrom_d16,volumefrom_d17,volumefrom_d18,volumefrom_d19,volumefrom_d20,volumefrom_d21,volumefrom_d22,volumefrom_d23,volumefrom_d24,volumefrom_d25,volumefrom_d26,volumefrom_d27,volumefrom_d28,volumefrom_d29,volumefrom_d30,volumefrom_d31,volumeto_d0,volumeto_d1,volumeto_d2,volumeto_d3,volumeto_d4,volumeto_d5,volumeto_d6,volumeto_d7,volumeto_d8,volumeto_d9,volumeto_d10,volumeto_d11,volumeto_d12,volumeto_d13,volumeto_d14,volumeto_d15,volumeto_d16,volumeto_d17,volumeto_d18,volumeto_d19,volumeto_d20,volumeto_d21,volumeto_d22,volumeto_d23,volumeto_d24,volumeto_d25,volumeto_d26,volumeto_d27,volumeto_d28,volumeto_d29,volumeto_d30,volumeto_d31
0,https://t.me/tornadosignals,Tornado Signals,TEC,Yobit,1532115901,2018-07-20T19:45:01.689,2018-07-20 19:45:01+00:00,TEC,0.095975,0.095975,0.095975,0.095975,0.108359,0.108359,0.108359,0.185759,0.185759,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.746130,0.746130,0.746130,0.746130,0.746130,0.356037,0.356037,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.040312,0.040312,0.040312,0.045514,0.045514,0.045514,0.078023,0.078023,0.078023,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.313394,0.313394,0.313394,0.313394,0.313394,0.313394,0.149545,0.426528,0.420026,0.420026,0.420026,0.420026,0.420026,0.092879,0.095975,0.095975,0.095975,0.095975,0.108359,0.108359,0.108359,0.185759,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.746130,0.746130,0.746130,0.746130,0.356037,0.356037,0.356037,1.000000,1.000000,1.000000,1.000000,1.000000,0.284830,0.095975,0.095975,0.095975,0.095975,0.108359,0.108359,0.108359,0.185759,0.185759,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.746130,0.746130,0.746130,0.746130,0.746130,0.356037,0.356037,1.0000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.00000,0.001646,0.000000,0.000000,0.002004,0.000000,0.003680,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.042565,0.000000,0.000000,0.00000,0.000000,0.003048,0.000000,0.027201,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000682,0.000000,0.000000,0.000845,0.000000,0.001674,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.019740,0.000000,0.000000,0.000000,0.000000,0.002163,0.000000,0.020447,0.000000,0.000000,0.00000,0.000000,0.000000
1,https://t.me/tornadosignals,Tornado Signals,TEC,Yobit,1532115901,2018-07-20T

In [14]:
d_scaled.head(10)[['pumptime_edited','currency','exchange','coin_name','pumped_yn']]

,pumptime_edited,currency,exchange,coin_name,pumped_yn
0,2018-07-20 19:45:01+00:00,TEC,Yobit,TEC,1
1,2018-07-20 19:45:01+00:00,TEC,Yobit,BCH,0
2,2018-07-20 19:45:01+00:00,TEC,Yobit,BAT,0
3,2018-07-20 19:45:01+00:00,TEC,Yobit,TKN,0
4,2018-07-20 19:45:01+00:00,TEC,Yobit,SNT,0
5,2018-07-20 19:45:01+00:00,TEC,Yobit,MET,0
6,2018-07-20 19:45:01+00:00,TEC,Yobit,PIVX,0
7,2018-07-20 19:45:01+00:00,TEC,Yobit,ZIL,0
8,2018-07-20 19:45:01+00:00,TEC,Yobit,QKC,0
9,2018-07-20 19:45:01+00:00,TEC,Yobit,BTS,0


In [15]:
d_scaled.to_pickle("ohlcv_data_slide_scaled_d.pkl")